In [2]:
import os
import csv

#reads all the episodes in a particular folder and reutns them in a array consisting of arrays with tuples
def read_Episodes(baseFolder):
    episodes = []
    for filename in sorted(os.listdir(baseFolder)):
        filePath = os.path.join(baseFolder, filename)
        episode = []
        with open(filePath, "r", newline="") as file:
            reader = csv.reader(file)
            for row in reader:
                observation = [float(epi) for epi in row]
                episode.append(observation)
            episodes.append(episode)        
    return episodes

In [7]:
#ToDO
#Termin mit Graf finden
#reale Daten mit in den Testdatensatz einfügen
#die time deiff eval episoden auch aus den realen Daten holen
#batch size mal riesig wählen mit 1000 und vieeleicht 2500(10%)
#lutz anschreiben mehr wetterstationen
#Tag als Episoden länge testen

In [9]:
import d3rlpy
import numpy as np
import os

EPIfolder = "../data/episFormula/3. corrected sign"

epis = read_Episodes(EPIfolder)

observations = []
actions = []
rewards = []
terminals = []

for epi in epis:
    for observation in epi:
        observations.append(observation[:-2])
        actions.append(observation[-2:-1])
        rewards.append(observation[-1])
    terminals += [0] * (len(epi) - 1) + [1]

observations=np.array(observations)
actions=np.array(actions)
rewards=np.array(rewards)
terminals=np.array(terminals)

print(observations.shape)

dataset = d3rlpy.dataset.MDPDataset(
    action_space=d3rlpy.constants.ActionSpace.CONTINUOUS,
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)


# create action scaler
action_scaler = d3rlpy.preprocessing.MinMaxActionScaler()
observation_scaler = d3rlpy.preprocessing.MinMaxObservationScaler()

# if you don't use GPU, set device=None instead.
rlModel = d3rlpy.algos.PLASConfig(action_scaler=action_scaler, observation_scaler=observation_scaler, batch_size=2500).create(device='cuda:0')

# combine FileAdapterFactory and TensorboardAdapterFactory
logger_adapter = d3rlpy.logging.CombineAdapterFactory([
   d3rlpy.logging.FileAdapterFactory(root_dir="d3rlpy_logs"),
   d3rlpy.logging.TensorboardAdapterFactory(root_dir="tensorboard_logs"),
])

# use partial episodes as test data
test_episodes = dataset.episodes[:200]

result = rlModel.fit(
    dataset,
    n_steps=500000,
    n_steps_per_epoch=1000,
    evaluators={
        'td_error': d3rlpy.metrics.TDErrorEvaluator(test_episodes),
        'value_scale': d3rlpy.metrics.AverageValueEstimationEvaluator(test_episodes),
        'discounted_advantage': d3rlpy.metrics.DiscountedSumOfAdvantageEvaluator(test_episodes),
        'initial_state': d3rlpy.metrics.InitialStateValueEstimationEvaluator(test_episodes),
        'diff_eval': d3rlpy.metrics.ContinuousActionDiffEvaluator(test_episodes)
    },
    logger_adapter=logger_adapter,
)

(23981, 7)
2025-11-28 18:13.54 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-28 18:13.54 [info     ] Action size has been automatically determined. action_size=1
2025-11-28 18:13.54 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-11-28 18:13.54 [debug    ] Fitting observation scaler...  observation_scaler=min_max
2025-11-28 18:13.55 [debug    ] Fitting action scaler...       action_scaler=min_max
2025-11-28 18:13.55 [debug    ] Building models...            
2025-11-28 18:13.55

Epoch 1/500: 100%|██████████| 1000/1000 [01:02<00:00, 16.08it/s, vae_loss=0.0564]


2025-11-28 18:15.01 [info     ] PLAS_20251128181355: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.051406193017959595, 'time_algorithm_update': 0.009520612239837646, 'vae_loss': 0.0562640067525208, 'time_step': 0.061195477962493895, 'td_error': 0.4605041648802969, 'value_scale': -0.028648496159701314, 'discounted_advantage': -0.007599421476259652, 'initial_state': -0.03709198161959648, 'diff_eval': 2618.3255392954375} step=1000
2025-11-28 18:15.02 [info     ] Model parameters are saved to d3rlpy_logs\PLAS_20251128181355\model_1000.d3


Epoch 2/500: 100%|██████████| 1000/1000 [01:01<00:00, 16.16it/s, vae_loss=0.0375]


2025-11-28 18:16.07 [info     ] PLAS_20251128181355: epoch=2 step=2000 epoch=2 metrics={'time_sample_batch': 0.05150544619560242, 'time_algorithm_update': 0.009159546852111816, 'vae_loss': 0.037484830321744086, 'time_step': 0.06090391683578491, 'td_error': 0.460472743536781, 'value_scale': -0.028616194409526702, 'discounted_advantage': -0.00802703172191488, 'initial_state': -0.03714320808649063, 'diff_eval': 2219.325241304062} step=2000
2025-11-28 18:16.08 [info     ] Model parameters are saved to d3rlpy_logs\PLAS_20251128181355\model_2000.d3


...

2025-11-29 04:16.41 [info     ] PLAS_20251128181355: epoch=498 step=498000 epoch=498 metrics={'time_sample_batch': 0.05177708625793457, 'time_algorithm_update': 0.008778275728225707, 'vae_loss': 0.0024075850383378566, 'time_step': 0.06080767011642456, 'td_error': 0.45927137829408166, 'value_scale': -0.029770637692981973, 'discounted_advantage': 6.240610869914742e-05, 'initial_state': -0.038178056478500366, 'diff_eval': 81.69310553708826} step=498000
2025-11-29 04:16.41 [info     ] Model parameters are saved to d3rlpy_logs\PLAS_20251128181355\model_498000.d3


Epoch 499/500: 100%|██████████| 1000/1000 [01:23<00:00, 11.99it/s, vae_loss=0.0024]


2025-11-29 04:18.09 [info     ] PLAS_20251128181355: epoch=499 step=499000 epoch=499 metrics={'time_sample_batch': 0.06802935600280761, 'time_algorithm_update': 0.01203147315979004, 'vae_loss': 0.002396826873649843, 'time_step': 0.08034662199020386, 'td_error': 0.4592392862677987, 'value_scale': -0.029819735191600073, 'discounted_advantage': 0.00023778548129319805, 'initial_state': -0.038354553282260895, 'diff_eval': 81.94097694957239} step=499000
2025-11-29 04:18.09 [info     ] Model parameters are saved to d3rlpy_logs\PLAS_20251128181355\model_499000.d3


Epoch 500/500: 100%|██████████| 1000/1000 [01:13<00:00, 13.68it/s, vae_loss=0.00242]


2025-11-29 04:19.29 [info     ] PLAS_20251128181355: epoch=500 step=500000 epoch=500 metrics={'time_sample_batch': 0.05976439642906189, 'time_algorithm_update': 0.012111942529678345, 'vae_loss': 0.002417733816895634, 'time_step': 0.07212218737602234, 'td_error': 0.45919863989943754, 'value_scale': -0.0298435115946296, 'discounted_advantage': 0.0007560081495641868, 'initial_state': -0.03832617774605751, 'diff_eval': 83.18531206721241} step=500000
2025-11-29 04:19.29 [info     ] Model parameters are saved to d3rlpy_logs\PLAS_20251128181355\model_500000.d3
